In [1]:
def expand(R):
    # R= abcd
    return R[2] + R[3] + R[1] + R[2] + R[1] + R[0]

def shiftLeft(b, k):
    #Dịch vòng k bit sang trái
    r=b[k:]
    r= r + b[:k]
    return r

def XOR(a, b):
    assert len(a) == len(b), 'Không cùng số bit !'
    return ''.join(['0' if a[i] == b[i] else '1' for i in range(len(a))])

def compress(k):
    # Nén khóa từ 8 bit thành 6 bit
    return k[5] + k[1] + k[3] + k[2] + k[7] + k[0]

def genKey(key):
    # Sinh 3 khóa con 6bit từ khóa 8bit
    l, r= key[:4], key[4:]
    #print(l, r)
    keys= []
    for i in range(3):
        #print('Bước %d' % (i+1))
        if i % 2 == 0:
            l= shiftLeft(l, 1)
            r= shiftLeft(r, 1)
        else:
            l= shiftLeft(l, 2)
            r= shiftLeft(r, 2)
        #print(l, r)
        keys.append(compress(l+r))
    return keys

def SBox(b):
    # Thay thế
    s=[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
       [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
       [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
       [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]]
    row= int(b[0] + b[5], 2)
    col= int(b[1:-1], 2)
    r= "{0:b}".format(s[row][col])
    r= '0'*(4- len(r)) + r
    return r

def PBox(b):
    # Hoán vị 4bit
    return b[2] + b[0] + b[3] + b[1] # 1001 -> 0110 -> 1001

def F(R, K):
    #Feistel
    return PBox(SBox(XOR(expand(R), K)))

def encode(P, K):
    Pl, Pr= P[:4], P[4:]
    #print('Pl= %s, Pr= %s' % (Pl, Pr))
    key= genKey(K)
    #print('Sinh khóa con: ')
#     for i in range(3):
#         print('K%d= ' % (i+1) + key[i])
    for i in range(3):
        #print('Vòng %d:' % (i + 1))
        temp= Pr
        f= F(Pr, key[i])
        #print('F%d= %s' % ((i+1), f))
        Pr= XOR(f, Pl) 
        Pl= temp
        #print('L%d= %s, R%d= %s' % ((i+1), Pl,(i+1), Pr))
    return Pl + Pr

def decode(C, K):
    Cl, Cr= C[:4], C[4:]
    #print('Cl= %s, Cr= %s' % (Cl, Cr))
    key= genKey(K)
    #print('Sinh khóa con: ')
#     for i in range(3):
#         print('K%d= ' % (i+1) + key[i])
    for i in range(3):
        #print('Vòng %d:' % (i + 1))
        temp= Cl
        f= F(Cl, key[3-i-1])
        #print('F%d= %s' % ((i+1), f))
        Cl= XOR(f, Cr) 
        Cr= temp
        #print('L%d= %s, R%d= %s' % ((i+1), Cl,(i+1), Cr))
    return Cl + Cr

def textToBin(text): #16bit
    out= ''
    for letter in text:
        b= format(ord(letter), 'b')
        b= '0'*(16-len(b)) + b
        out += b
    return out

def binToText(btext): #16bit
    out= ''
    for i in range(0,len(btext), 16):
        b= btext[i: i+ 16]
        letter= chr(int(b, 2))
        out+= letter
    return out

def getBlocks(text, nbit= 4):   
    btext= textToBin(text)
    blocks = []
    for i in range(0, len(btext), nbit):
        blocks.append(btext[i:i+nbit])
    return blocks

def ENCODE(p, key):
    blocks= getBlocks(p, 8)
    enblocks= [encode(block, key) for block in blocks]
    c= ''.join(enblocks)
    return binToText(c)

def DECODE(c, key):
    blocks= getBlocks(c, 8)
    deblocks= [decode(block, key) for block in blocks]
    p= ''.join(deblocks)
    return binToText(p)

In [3]:
encode('01011100', '10011010')

'10010010'

In [59]:
decode('10010010', '10011010')

'01011100'

In [66]:
p= 'Hôm nay đi chơi đi !'
k= '10011010'

In [65]:
ENCODE(p, k)r

'ͧΪͱΤ͉εͽΤפֿπΤϜ϶\ufb45πΤפֿπΤϹ'

In [67]:
DECODE(ENCODE(p, k), k)

'Hôm nay đi chơi đi !'

In [2]:
def GCD_extended(x, y):
    u1, u2, u3 = 1, 0, x
    v1, v2, v3 = 0, 1, y
    while v3 != 0:
        q = u3//v3
        t1, t2, t3 = u1 - q*v1, u2 - q*v2, u3 - q*v3
        u1, u2, u3 = v1, v2, v3
        v1, v2, v3 = t1, t2, t3
    a, b, c= u1, u2, u3
    return a, b, c

In [30]:
def getPrivateKey(p, q, e):
    n= p*q
    print('n= %d' % n)
    phi= (p-1)*(q-1)
    print('phi= %d' % phi)
    d = GCD_extended(e,phi)[0]
    if d < 0:
        d+= phi
    print('d= %d' % d)
    return n, d

def E(M, k):
    n, x= k
    C= pow(M, x, n)
    return C

def D(C, k):
    n, x= k
    M= pow(C, x, n)
    return M

In [31]:
n1, d1= getPrivateKey(7, 11, 17)
e1= 17
n1, d1

n= 77
phi= 60
d= 53


(77, 53)

In [32]:
n2, d2= getPrivateKey(11,13,11)
e2= 11
n2, d2

n= 143
phi= 120
d= 11


(143, 11)

In [33]:
M= 9
C1= E(M, (n1, d1))
C= E(C1, (n2, e2))
C1, C

(25, 25)

In [34]:
pow(25, 11, 143)

25

In [35]:
C1= D(C, (n2, d2))
M= D(C1, (n1, e1))
C1, M

(25, 9)

In [87]:
c= pow(m, 3, n)

In [88]:
p= pow(c, 27, n)

In [89]:
p

4

In [90]:
c**27

58149737003040059690390169

In [91]:
9**3

729

In [92]:
pow(9,3,55)

14

In [93]:
pow(14,3,55)

49

In [94]:
pow(49,3,55)

4

In [98]:
pow(4,27,55)

49

In [96]:
27/2

13.5

In [97]:
13/2

6.5